Database Connection

In [18]:
import sqlite3

conn = sqlite3.connect('cancer.db')

try:
    conn.cursor()
    print("DB Connected");
except Exception as ex:
    print("DB not Connected");

DB Connected


FOLLOWING TABLES ARE CREATED -
User Table : This table contains the details of the user, who have tweeted
Cancer Type Table : This table holds different categories of cancer
Treatment Table: This table contains types of treatment based on Cancer type
Location Table:This table describes the location of the user

In [19]:
#USER TABLE
conn = sqlite3.connect('cancer.db')
print("Opened database successfully");
conn.execute('''DROP TABLE IF EXISTS USER;''')
conn.execute('''CREATE TABLE IF NOT EXISTS USER
         (
         User_Id INTEGER PRIMARY KEY AUTOINCREMENT,
         User_Name     TEXT   NOT NULL,
         Handle_Name   TEXT   NOT NULL,
         Description   TEXT   NOT NULL,
         Followers_Count     INTEGER,
         Created_Date   TEXT  NOT NULL,
         Image_Url      TEXT  NOT NULL,
         Smoker      TEXT  NOT NULL,
         hasCancer      TEXT  NOT NULL,
         Cancer_Type      TEXT  NOT NULL
         );''')

print("User Table created successfully");

conn.close()

#CANCER_TYPE TABLE
conn = sqlite3.connect('cancer.db')
print("Opened database successfully");
conn.execute('''DROP TABLE IF EXISTS CANCER_TYPE;''')
conn.execute('''CREATE TABLE IF NOT EXISTS CANCER_TYPE
         (Hashtag_Id INTEGER NOT NULL,
         Cancer_Id INTEGER PRIMARY KEY AUTOINCREMENT,
         Cancer_Type      TEXT  NOT NULL,
         FOREIGN KEY(Hashtag_Id) REFERENCES HASHTAG(Hashtag_Id));''')

print("Cancer_Type Table created successfully");

conn.close()

#TREATMENT TABLE
conn = sqlite3.connect('cancer.db')
print("Opened database successfully");
conn.execute('''DROP TABLE IF EXISTS TREATMENT;''')
conn.execute('''CREATE TABLE IF NOT EXISTS TREATMENT
         (Cancer_Id INTEGER NOT NULL,
          Treatment_Id INTEGER PRIMARY KEY AUTOINCREMENT,
          Treatment_Type      TEXT  NOT NULL,
          FOREIGN KEY(Cancer_Id) REFERENCES CANCER_TYPE(Cancer_Id));''')

print("Treatment Table created successfully");

conn.close()

#LOCATION TABLE
conn = sqlite3.connect('cancer.db')
print("Opened database successfully");
conn.execute('''DROP TABLE IF EXISTS LOCATION;''')
conn.execute('''CREATE TABLE IF NOT EXISTS LOCATION
         (User_Id INTEGER NOT NULL ,
          Location_Id INTEGER PRIMARY KEY AUTOINCREMENT,
          Location    TEXT  ,
          FOREIGN KEY(User_Id) REFERENCES USER(User_Id) );''')

print("Location Table created successfully");

conn.close()



Opened database successfully
User Table created successfully
Opened database successfully
Cancer_Type Table created successfully
Opened database successfully
Treatment Table created successfully
Opened database successfully
Location Table created successfully


FOLLOWING TABLES ARE CREATED -
Tweets Table : This table contains the details of the tweet with the tweet handle name and the posted date
Hashtag Table : This table holds details on the hashtag and the tweet_id
Tweet mention Table: This table contains information about the tweet mention, source user and target user
Tweet URL Table: This table describes the URL of the tweet and the URL Id

In [20]:
#TWEETS TABLE
conn = sqlite3.connect('cancer.db')
print("Opened database successfully");
conn.execute('''DROP TABLE IF EXISTS TWEETS;''')
conn.execute('''CREATE TABLE IF NOT EXISTS TWEETS
         (Tweet_Id INTEGER PRIMARY KEY NOT NULL,
          Tweet_Handle TEXT NOT NULL,
          Tweet_Text TEXT NOT NULL,
          Posted_At TEXT,
          Alert TEXT,
          User_Id INTEGER NOT NULL,
          FOREIGN KEY(User_Id) REFERENCES USER(User_Id));''')

print("Tweets Table created successfully");

conn.close()

#HASHTAG TABLE
conn = sqlite3.connect('cancer.db')
print("Opened database successfully");
conn.execute('''DROP TABLE IF EXISTS HASHTAG;''')
conn.execute('''CREATE TABLE IF NOT EXISTS HASHTAG
         (Hashtag_Id INTEGER PRIMARY KEY NOT NULL,
          Hashtag_Text TEXT NOT NULL,
          Tweet_Id  INTEGER NOT NULL,
          FOREIGN KEY(Tweet_Id) REFERENCES TWEETS(Tweet_Id));''')

print("Hashtag Table created successfully");

conn.close()

#TWEET MENTION TABLE
conn = sqlite3.connect('cancer.db')
print("Opened database successfully");
conn.execute('''DROP TABLE IF EXISTS TWEET_MENTION;''')
conn.execute('''CREATE TABLE IF NOT EXISTS TWEET_MENTION
         (Mention_Id INTEGER PRIMARY KEY NOT NULL,
          Source_User TEXT NOT NULL,
          Target_User TEXT NOT NULL,
          Tweet_Id  INTEGER NOT NULL,
          FOREIGN KEY(Tweet_Id) REFERENCES TWEETS(Tweet_Id));''')

print("Tweet Mention Table created successfully");

conn.close()

#TWEET URL TABLE
conn = sqlite3.connect('cancer.db')
print("Opened database successfully");
conn.execute('''DROP TABLE IF EXISTS TWEET_URL;''')
conn.execute('''CREATE TABLE IF NOT EXISTS TWEET_URL
         (Url_Id INTEGER PRIMARY KEY NOT NULL,
          Tweet_Url TEXT NOT NULL,
          Tweet_Id  INTEGER NOT NULL,
          FOREIGN KEY(Tweet_Id) REFERENCES TWEETS(Tweet_Id));''')

print("Tweet URL Table created successfully");

conn.close()


Opened database successfully
Tweets Table created successfully
Opened database successfully
Hashtag Table created successfully
Opened database successfully
Tweet Mention Table created successfully
Opened database successfully
Tweet URL Table created successfully


Scrapping Twitter for #breastcancer

In [21]:
from twitter_scraper_selenium import scrape_keyword
import json
from twitter_scraper_selenium import get_profile_details
import random

CANCER_TYPE = 'breastcancer'

#SCRAPING FOR CANCER KEYWORD
cancer = json.loads(scrape_keyword(keyword='#' + CANCER_TYPE, browser="chrome",
                                   tweets_count=100, output_format="json", until="2023-08-31", since="2021-11-11"))

for item in cancer:
    userName = cancer[item]['username'];

    #Fetching User profile Data
    filename = "twitter_api_data"
    data = json.loads(get_profile_details(twitter_username=userName))

    #Storing details which are to be inserted
    Handle_Name = data['name']
    Description = data['description']
    Followers_Count = data['followers_count']
    Created_Date = data['created_at']
    smoker = random.choice(['Yes', 'No'])
    hasCancer = random.choice(['Yes', 'No'])
    Image_Url = data['profile_image_url']
    Tweets_Handle = data['name']
    Tweets_Text = cancer[item]['content']
    Posted_At = cancer[item]['posted_time']
    Tweet_id = cancer[item]['tweet_id']
    location = 'NA' if data['location'] == "" else data['location']
    Tweet_Url = cancer[item]['tweet_url']
    alert = random.choice(['Yes', 'No'])

    #Insert into USER table
    conn = sqlite3.connect('cancer.db')
    conn.execute(
        'INSERT INTO USER (User_Name,Handle_Name,Description,Followers_Count,Created_Date,Image_Url,Smoker,hasCancer,Cancer_Type) VALUES (?,?,?,?,?,?,?,?,?)',
        (userName, Handle_Name, Description, Followers_Count, Created_Date, Image_Url, smoker, hasCancer, CANCER_TYPE))
    conn.commit()
    conn.close()

    #Fetch User_id from USER TABLE

    User_id = 0;
    conn = sqlite3.connect('cancer.db')
    cur = conn.cursor()
    cur.execute('SELECT User_Id FROM USER WHERE User_Name = ?', (userName,))
    rows = cur.fetchone()


    for row in rows:
        User_id = row

    conn.commit()
    conn.close()

    #Insert into TWEETS TABLE
    conn = sqlite3.connect('cancer.db')
    conn.execute(
        'INSERT INTO TWEETS (Tweet_Id,Tweet_Handle,Tweet_Text,Posted_At,Alert,User_Id) VALUES (?,?,?,?,?,?)',
        (Tweet_id, Handle_Name, Tweets_Text, Posted_At, alert, User_id))
    conn.commit()
    conn.close()

    #Insert into LOCATION TABLE
    conn = sqlite3.connect('cancer.db')
    conn.execute(
        'INSERT INTO LOCATION (Location,User_Id) VALUES (?,?)',
        (location, User_id))
    conn.commit()
    conn.close()

    #Insert into TWEET_URL TABLE
    conn = sqlite3.connect('cancer.db')
    conn.execute(
        'INSERT INTO Tweet_Url (Tweet_Id,Tweet_Url) VALUES (?,?)',
        (Tweet_id, Tweet_Url))
    conn.commit()
    conn.close()

    #Insert into TWEET_MENTION TABLE
    conn = sqlite3.connect('cancer.db')
    for mention in cancer[item]['mentions']:
        conn.execute(
            'INSERT INTO TWEET_MENTION (Source_User,Target_User,Tweet_id) VALUES (?,?,?)',
            (Handle_Name, mention, Tweet_id))
        conn.commit()
    conn.close()

    #Insert into HASHTAG TABLE
    conn = sqlite3.connect('cancer.db')
    for text in cancer[item]['hashtags']:
        conn.execute(
            'INSERT INTO HASHTAG (Hashtag_Text,Tweet_id) VALUES (?,?)',
            (text, Tweet_id))
        conn.commit()
    conn.close()

    #Fetch Hashtag_id from USER TABLE

    Hashtag_id = 0;
    conn = sqlite3.connect('cancer.db')
    cur = conn.cursor()
    cur.execute('SELECT Hashtag_id FROM HASHTAG WHERE Tweet_Id = ? and Hashtag_Text=? collate nocase',
                (Tweet_id, CANCER_TYPE))
    rows = cur.fetchone()

    if rows != None:
        for row in rows:
            Hashtag_id = row

    conn.commit()
    conn.close()

    #Insert into CANCER_TYPE TABLE
    conn = sqlite3.connect('cancer.db')
    conn.execute('INSERT INTO CANCER_TYPE (Hashtag_Id,Cancer_type) VALUES (?,?)', (Hashtag_id, CANCER_TYPE))
    conn.commit()
    conn.close()

    #Fetch Cancer_Id from CANCER_TYPE TABLE
    Cancer_Id = 0;
    conn = sqlite3.connect('cancer.db')
    cur = conn.cursor()
    cur.execute('SELECT Cancer_Id FROM CANCER_TYPE WHERE CANCER_TYPE=?', (CANCER_TYPE,))
    rows = cur.fetchone()

    if rows != None:
        for row in rows:
            Cancer_Id = row

    conn.commit()
    conn.close()

    #Insert into TREATMENT TABLE
    conn = sqlite3.connect('cancer.db')
    treatment = ['Surgery', 'Radiation', 'Chemotherapy']
    conn.execute('INSERT INTO TREATMENT (Cancer_Id,Treatment_type) VALUES (?,?)', (Cancer_Id, random.choice(treatment)))

    conn.commit()
    conn.close()
print("Twitter Data added successfully !")


[WDM] - Current google-chrome version is 107.0.5304
[WDM] - Get LATEST driver version for 107.0.5304


[WDM] - Driver [/Users/amulyamurahari/.wdm/drivers/chromedriver/mac64/107.0.5304.62/chromedriver] found in cache


Twitter Data added successfully !


Scrapping Twitter for #lungcancer

In [22]:
from twitter_scraper_selenium import scrape_keyword
import json
from twitter_scraper_selenium import get_profile_details
import random

CANCER_TYPE = 'lungcancer'

#SCRAPING FOR CANCER KEYWORD
cancer = json.loads(scrape_keyword(keyword='#' + CANCER_TYPE, browser="chrome",
                                   tweets_count=70, output_format="json", until="2023-08-31", since="2021-11-11"))

for item in cancer:
    userName = cancer[item]['username'];

    #Fetching User profile Data
    filename = "twitter_api_data"
    data = json.loads(get_profile_details(twitter_username=userName))

    #Storing details which are to be inserted
    Handle_Name = data['name']
    Description = data['description']
    Followers_Count = data['followers_count']
    Created_Date = data['created_at']
    smoker = random.choice(['Yes', 'No'])
    hasCancer = random.choice(['Yes', 'No'])
    Image_Url = data['profile_image_url']
    Tweets_Handle = data['name']
    Tweets_Text = cancer[item]['content']
    Posted_At = cancer[item]['posted_time']
    Tweet_id = cancer[item]['tweet_id']
    location = 'NA' if data['location'] == "" else data['location']
    Tweet_Url = cancer[item]['tweet_url']
    alert = random.choice(['Yes', 'No'])

    #Insert into USER table
    conn = sqlite3.connect('cancer.db')
    conn.execute(
        'INSERT INTO USER (User_Name,Handle_Name,Description,Followers_Count,Created_Date,Image_Url,Smoker,hasCancer,Cancer_Type) VALUES (?,?,?,?,?,?,?,?,?)',
        (userName, Handle_Name, Description, Followers_Count, Created_Date, Image_Url, smoker, hasCancer, CANCER_TYPE))
    conn.commit()
    conn.close()

    #Fetch User_id from USER TABLE

    User_id = 0;
    conn = sqlite3.connect('cancer.db')
    cur = conn.cursor()
    cur.execute('SELECT User_Id FROM USER WHERE User_Name = ?', (userName,))
    rows = cur.fetchone()


    for row in rows:
        User_id = row

    conn.commit()
    conn.close()

    #Insert into TWEETS TABLE
    conn = sqlite3.connect('cancer.db')
    conn.execute(
        'INSERT INTO TWEETS (Tweet_Id,Tweet_Handle,Tweet_Text,Posted_At,Alert,User_Id) VALUES (?,?,?,?,?,?)',
        (Tweet_id, Handle_Name, Tweets_Text, Posted_At, alert, User_id))
    conn.commit()
    conn.close()

    #Insert into LOCATION TABLE
    conn = sqlite3.connect('cancer.db')
    conn.execute(
        'INSERT INTO LOCATION (Location,User_Id) VALUES (?,?)',
        (location, User_id))
    conn.commit()
    conn.close()

    #Insert into TWEET_URL TABLE
    conn = sqlite3.connect('cancer.db')
    conn.execute(
        'INSERT INTO Tweet_Url (Tweet_Id,Tweet_Url) VALUES (?,?)',
        (Tweet_id, Tweet_Url))
    conn.commit()
    conn.close()

    #Insert into TWEET_MENTION TABLE
    conn = sqlite3.connect('cancer.db')
    for mention in cancer[item]['mentions']:
        conn.execute(
            'INSERT INTO TWEET_MENTION (Source_User,Target_User,Tweet_id) VALUES (?,?,?)',
            (Handle_Name, mention, Tweet_id))
        conn.commit()
    conn.close()

    #Insert into HASHTAG TABLE
    conn = sqlite3.connect('cancer.db')
    for text in cancer[item]['hashtags']:
        conn.execute(
            'INSERT INTO HASHTAG (Hashtag_Text,Tweet_id) VALUES (?,?)',
            (text, Tweet_id))
        conn.commit()
    conn.close()

    #Fetch Hashtag_id from USER TABLE

    Hashtag_id = 0;
    conn = sqlite3.connect('cancer.db')
    cur = conn.cursor()
    cur.execute('SELECT Hashtag_id FROM HASHTAG WHERE Tweet_Id = ? and Hashtag_Text=? collate nocase',
                (Tweet_id, CANCER_TYPE))
    rows = cur.fetchone()

    if rows != None:
        for row in rows:
            Hashtag_id = row

    conn.commit()
    conn.close()

    #Insert into CANCER_TYPE TABLE
    conn = sqlite3.connect('cancer.db')
    conn.execute('INSERT INTO CANCER_TYPE (Hashtag_Id,Cancer_type) VALUES (?,?)', (Hashtag_id, CANCER_TYPE))
    conn.commit()
    conn.close()

    #Fetch Cancer_Id from CANCER_TYPE TABLE
    Cancer_Id = 0;
    conn = sqlite3.connect('cancer.db')
    cur = conn.cursor()
    cur.execute('SELECT Cancer_Id FROM CANCER_TYPE WHERE CANCER_TYPE=?', (CANCER_TYPE,))
    rows = cur.fetchone()

    if rows != None:
        for row in rows:
            Cancer_Id = row

    conn.commit()
    conn.close()

    #Insert into TREATMENT TABLE
    conn = sqlite3.connect('cancer.db')
    treatment = ['Surgery', 'Radiation', 'Chemotherapy']
    conn.execute('INSERT INTO TREATMENT (Cancer_Id,Treatment_type) VALUES (?,?)', (Cancer_Id, random.choice(treatment)))

    conn.commit()
    conn.close()
print("Twitter Data added successfully !")



[WDM] - Current google-chrome version is 107.0.5304
[WDM] - Get LATEST driver version for 107.0.5304
[WDM] - Driver [/Users/amulyamurahari/.wdm/drivers/chromedriver/mac64/107.0.5304.62/chromedriver] found in cache


2022-11-12 18:35:37,982 - twitter_scraper_selenium.element_finder - ERROR - Error at method find_images : Message: no such element: Unable to locate element: {"method":"tag name","selector":"img"}
  (Session info: headless chrome=107.0.5304.110)
Stacktrace:
0   chromedriver                        0x00000001050532c8 chromedriver + 4752072
1   chromedriver                        0x0000000104fd3463 chromedriver + 4228195
2   chromedriver                        0x0000000104c36b18 chromedriver + 441112
3   chromedriver                        0x0000000104c73e21 chromedriver + 691745
4   chromedriver                        0x0000000104c74061 chromedriver + 692321
5   chromedriver                        0x0000000104c677b7 chromedriver + 640951
6   chromedriver                        0x0000000104c94d2d chromedriver + 826669
7   chromedriver                        0x0000000104c67685 chromedriver + 640645
8   chromedriver                        0x0000000104c94e5e chromedriver + 826974
9   chromed

Twitter Data added successfully !


Scraping Twitter for #bloodcancer

In [23]:
from twitter_scraper_selenium import scrape_keyword
import json
from twitter_scraper_selenium import get_profile_details
import random

CANCER_TYPE = 'bloodcancer'

#SCRAPING FOR CANCER KEYWORD
cancer = json.loads(scrape_keyword(keyword='#' + CANCER_TYPE, browser="chrome",
                                   tweets_count=80, output_format="json", until="2023-08-31", since="2021-11-11"))

for item in cancer:
    userName = cancer[item]['username'];

    #Fetching User profile Data
    filename = "twitter_api_data"
    data = json.loads(get_profile_details(twitter_username=userName))

    #Storing details which are to be inserted
    Handle_Name = data['name']
    Description = data['description']
    Followers_Count = data['followers_count']
    Created_Date = data['created_at']
    smoker = random.choice(['Yes', 'No'])
    hasCancer = random.choice(['Yes', 'No'])
    Image_Url = data['profile_image_url']
    Tweets_Handle = data['name']
    Tweets_Text = cancer[item]['content']
    Posted_At = cancer[item]['posted_time']
    Tweet_id = cancer[item]['tweet_id']
    location = 'NA' if data['location'] == "" else data['location']
    Tweet_Url = cancer[item]['tweet_url']
    alert = random.choice(['Yes', 'No'])

    #Insert into USER table
    conn = sqlite3.connect('cancer.db')
    conn.execute(
        'INSERT INTO USER (User_Name,Handle_Name,Description,Followers_Count,Created_Date,Image_Url,Smoker,hasCancer,Cancer_Type) VALUES (?,?,?,?,?,?,?,?,?)',
        (userName, Handle_Name, Description, Followers_Count, Created_Date, Image_Url, smoker, hasCancer, CANCER_TYPE))
    conn.commit()
    conn.close()

    #Fetch User_id from USER TABLE

    User_id = 0;
    conn = sqlite3.connect('cancer.db')
    cur = conn.cursor()
    cur.execute('SELECT User_Id FROM USER WHERE User_Name = ?', (userName,))
    rows = cur.fetchone()


    for row in rows:
        User_id = row

    conn.commit()
    conn.close()

    #Insert into TWEETS TABLE
    conn = sqlite3.connect('cancer.db')
    conn.execute(
        'INSERT INTO TWEETS (Tweet_Id,Tweet_Handle,Tweet_Text,Posted_At,Alert,User_Id) VALUES (?,?,?,?,?,?)',
        (Tweet_id, Handle_Name, Tweets_Text, Posted_At, alert, User_id))
    conn.commit()
    conn.close()

    #Insert into LOCATION TABLE
    conn = sqlite3.connect('cancer.db')
    conn.execute(
        'INSERT INTO LOCATION (Location,User_Id) VALUES (?,?)',
        (location, User_id))
    conn.commit()
    conn.close()

    #Insert into TWEET_URL TABLE
    conn = sqlite3.connect('cancer.db')
    conn.execute(
        'INSERT INTO Tweet_Url (Tweet_Id,Tweet_Url) VALUES (?,?)',
        (Tweet_id, Tweet_Url))
    conn.commit()
    conn.close()

    #Insert into TWEET_MENTION TABLE
    conn = sqlite3.connect('cancer.db')
    for mention in cancer[item]['mentions']:
        conn.execute(
            'INSERT INTO TWEET_MENTION (Source_User,Target_User,Tweet_id) VALUES (?,?,?)',
            (Handle_Name, mention, Tweet_id))
        conn.commit()
    conn.close()

    #Insert into HASHTAG TABLE
    conn = sqlite3.connect('cancer.db')
    for text in cancer[item]['hashtags']:
        conn.execute(
            'INSERT INTO HASHTAG (Hashtag_Text,Tweet_id) VALUES (?,?)',
            (text, Tweet_id))
        conn.commit()
    conn.close()

    #Fetch Hashtag_id from USER TABLE

    Hashtag_id = 0;
    conn = sqlite3.connect('cancer.db')
    cur = conn.cursor()
    cur.execute('SELECT Hashtag_id FROM HASHTAG WHERE Tweet_Id = ? and Hashtag_Text=? collate nocase',
                (Tweet_id, CANCER_TYPE))
    rows = cur.fetchone()

    if rows != None:
        for row in rows:
            Hashtag_id = row

    conn.commit()
    conn.close()

    #Insert into CANCER_TYPE TABLE
    conn = sqlite3.connect('cancer.db')
    conn.execute('INSERT INTO CANCER_TYPE (Hashtag_Id,Cancer_type) VALUES (?,?)', (Hashtag_id, CANCER_TYPE))
    conn.commit()
    conn.close()

    #Fetch Cancer_Id from CANCER_TYPE TABLE
    Cancer_Id = 0;
    conn = sqlite3.connect('cancer.db')
    cur = conn.cursor()
    cur.execute('SELECT Cancer_Id FROM CANCER_TYPE WHERE CANCER_TYPE=?', (CANCER_TYPE,))
    rows = cur.fetchone()

    if rows != None:
        for row in rows:
            Cancer_Id = row

    conn.commit()
    conn.close()

    #Insert into TREATMENT TABLE
    conn = sqlite3.connect('cancer.db')
    treatment = ['Surgery', 'Radiation', 'Chemotherapy']
    conn.execute('INSERT INTO TREATMENT (Cancer_Id,Treatment_type) VALUES (?,?)', (Cancer_Id, random.choice(treatment)))

    conn.commit()
    conn.close()
print("Twitter Data added successfully !")

[WDM] - Current google-chrome version is 107.0.5304
[WDM] - Get LATEST driver version for 107.0.5304


[WDM] - Driver [/Users/amulyamurahari/.wdm/drivers/chromedriver/mac64/107.0.5304.62/chromedriver] found in cache
2022-11-12 18:39:17,622 - twitter_scraper_selenium.driver_utils - ERROR - Tweets did not appear!, Try setting headless=False to see what is happening
Traceback (most recent call last):
  File "/Users/amulyamurahari/Library/Python/3.9/lib/python/site-packages/twitter_scraper_selenium/driver_utils.py", line 35, in wait_until_tweets_appear
    WebDriverWait(driver, 10).until(EC.presence_of_element_located(
  File "/Users/amulyamurahari/Library/Python/3.9/lib/python/site-packages/selenium/webdriver/support/wait.py", line 90, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x0000000100b652c8 chromedriver + 4752072
1   chromedriver                        0x0000000100ae5463 chromedriver + 4228195
2   chromedriver                        0x0000000100748b18 chrom

KeyboardInterrupt: 

Scraping Twitter for #kidneycancer

In [24]:
from twitter_scraper_selenium import scrape_keyword
import json
from twitter_scraper_selenium import get_profile_details
import random

CANCER_TYPE = 'kidneycancer'

#SCRAPING FOR CANCER KEYWORD
cancer = json.loads(scrape_keyword(keyword='#' + CANCER_TYPE, browser="chrome",
                                   tweets_count=40, output_format="json", until="2023-08-31", since="2021-11-11"))

for item in cancer:
    userName = cancer[item]['username'];

    #Fetching User profile Data
    filename = "twitter_api_data"
    data = json.loads(get_profile_details(twitter_username=userName))

    #Storing details which are to be inserted
    Handle_Name = data['name']
    Description = data['description']
    Followers_Count = data['followers_count']
    Created_Date = data['created_at']
    smoker = random.choice(['Yes', 'No'])
    hasCancer = random.choice(['Yes', 'No'])
    Image_Url = data['profile_image_url']
    Tweets_Handle = data['name']
    Tweets_Text = cancer[item]['content']
    Posted_At = cancer[item]['posted_time']
    Tweet_id = cancer[item]['tweet_id']
    location = 'NA' if data['location'] == "" else data['location']
    Tweet_Url = cancer[item]['tweet_url']
    alert = random.choice(['Yes', 'No'])

    #Insert into USER table
    conn = sqlite3.connect('cancer.db')
    conn.execute(
        'INSERT INTO USER (User_Name,Handle_Name,Description,Followers_Count,Created_Date,Image_Url,Smoker,hasCancer,Cancer_Type) VALUES (?,?,?,?,?,?,?,?,?)',
        (userName, Handle_Name, Description, Followers_Count, Created_Date, Image_Url, smoker, hasCancer, CANCER_TYPE))
    conn.commit()
    conn.close()

    #Fetch User_id from USER TABLE

    User_id = 0;
    conn = sqlite3.connect('cancer.db')
    cur = conn.cursor()
    cur.execute('SELECT User_Id FROM USER WHERE User_Name = ?', (userName,))
    rows = cur.fetchone()


    for row in rows:
        User_id = row

    conn.commit()
    conn.close()

    #Insert into TWEETS TABLE
    conn = sqlite3.connect('cancer.db')
    conn.execute(
        'INSERT INTO TWEETS (Tweet_Id,Tweet_Handle,Tweet_Text,Posted_At,Alert,User_Id) VALUES (?,?,?,?,?,?)',
        (Tweet_id, Handle_Name, Tweets_Text, Posted_At, alert, User_id))
    conn.commit()
    conn.close()

    #Insert into LOCATION TABLE
    conn = sqlite3.connect('cancer.db')
    conn.execute(
        'INSERT INTO LOCATION (Location,User_Id) VALUES (?,?)',
        (location, User_id))
    conn.commit()
    conn.close()

    #Insert into TWEET_URL TABLE
    conn = sqlite3.connect('cancer.db')
    conn.execute(
        'INSERT INTO Tweet_Url (Tweet_Id,Tweet_Url) VALUES (?,?)',
        (Tweet_id, Tweet_Url))
    conn.commit()
    conn.close()

    #Insert into TWEET_MENTION TABLE
    conn = sqlite3.connect('cancer.db')
    for mention in cancer[item]['mentions']:
        conn.execute(
            'INSERT INTO TWEET_MENTION (Source_User,Target_User,Tweet_id) VALUES (?,?,?)',
            (Handle_Name, mention, Tweet_id))
        conn.commit()
    conn.close()

    #Insert into HASHTAG TABLE
    conn = sqlite3.connect('cancer.db')
    for text in cancer[item]['hashtags']:
        conn.execute(
            'INSERT INTO HASHTAG (Hashtag_Text,Tweet_id) VALUES (?,?)',
            (text, Tweet_id))
        conn.commit()
    conn.close()

    #Fetch Hashtag_id from USER TABLE

    Hashtag_id = 0;
    conn = sqlite3.connect('cancer.db')
    cur = conn.cursor()
    cur.execute('SELECT Hashtag_id FROM HASHTAG WHERE Tweet_Id = ? and Hashtag_Text=? collate nocase',
                (Tweet_id, CANCER_TYPE))
    rows = cur.fetchone()

    if rows != None:
        for row in rows:
            Hashtag_id = row

    conn.commit()
    conn.close()

    #Insert into CANCER_TYPE TABLE
    conn = sqlite3.connect('cancer.db')
    conn.execute('INSERT INTO CANCER_TYPE (Hashtag_Id,Cancer_type) VALUES (?,?)', (Hashtag_id, CANCER_TYPE))
    conn.commit()
    conn.close()

    #Fetch Cancer_Id from CANCER_TYPE TABLE
    Cancer_Id = 0;
    conn = sqlite3.connect('cancer.db')
    cur = conn.cursor()
    cur.execute('SELECT Cancer_Id FROM CANCER_TYPE WHERE CANCER_TYPE=?', (CANCER_TYPE,))
    rows = cur.fetchone()

    if rows != None:
        for row in rows:
            Cancer_Id = row

    conn.commit()
    conn.close()

    #Insert into TREATMENT TABLE
    conn = sqlite3.connect('cancer.db')
    treatment = ['Surgery', 'Radiation', 'Chemotherapy']
    conn.execute('INSERT INTO TREATMENT (Cancer_Id,Treatment_type) VALUES (?,?)', (Cancer_Id, random.choice(treatment)))

    conn.commit()
    conn.close()
print("Twitter Data added successfully !")

[WDM] - Current google-chrome version is 107.0.5304
[WDM] - Get LATEST driver version for 107.0.5304
[WDM] - Driver [/Users/amulyamurahari/.wdm/drivers/chromedriver/mac64/107.0.5304.62/chromedriver] found in cache


 
Twitter Data added successfully !


Scraping Twitter for #bladdercancer

In [ ]:
from twitter_scraper_selenium import scrape_keyword
import json
from twitter_scraper_selenium import get_profile_details
import random

CANCER_TYPE = 'bladdercancer'

#SCRAPING FOR CANCER KEYWORD
cancer = json.loads(scrape_keyword(keyword='#' + CANCER_TYPE, browser="chrome",
                                   tweets_count=50, output_format="json", until="2023-08-31", since="2021-11-11"))

for item in cancer:
    userName = cancer[item]['username'];

    #Fetching User profile Data
    filename = "twitter_api_data"
    data = json.loads(get_profile_details(twitter_username=userName))

    #Storing details which are to be inserted
    Handle_Name = data['name']
    Description = data['description']
    Followers_Count = data['followers_count']
    Created_Date = data['created_at']
    smoker = random.choice(['Yes', 'No'])
    hasCancer = random.choice(['Yes', 'No'])
    Image_Url = data['profile_image_url']
    Tweets_Handle = data['name']
    Tweets_Text = cancer[item]['content']
    Posted_At = cancer[item]['posted_time']
    Tweet_id = cancer[item]['tweet_id']
    location = 'NA' if data['location'] == "" else data['location']
    Tweet_Url = cancer[item]['tweet_url']
    alert = random.choice(['Yes', 'No'])

    #Insert into USER table
    conn = sqlite3.connect('cancer.db')
    conn.execute(
        'INSERT INTO USER (User_Name,Handle_Name,Description,Followers_Count,Created_Date,Image_Url,Smoker,hasCancer,Cancer_Type) VALUES (?,?,?,?,?,?,?,?,?)',
        (userName, Handle_Name, Description, Followers_Count, Created_Date, Image_Url, smoker, hasCancer, CANCER_TYPE))
    conn.commit()
    conn.close()

    #Fetch User_id from USER TABLE

    User_id = 0;
    conn = sqlite3.connect('cancer.db')
    cur = conn.cursor()
    cur.execute('SELECT User_Id FROM USER WHERE User_Name = ?', (userName,))
    rows = cur.fetchone()


    for row in rows:
        User_id = row

    conn.commit()
    conn.close()

    #Insert into TWEETS TABLE
    conn = sqlite3.connect('cancer.db')
    conn.execute(
        'INSERT INTO TWEETS (Tweet_Id,Tweet_Handle,Tweet_Text,Posted_At,Alert,User_Id) VALUES (?,?,?,?,?,?)',
        (Tweet_id, Handle_Name, Tweets_Text, Posted_At, alert, User_id))
    conn.commit()
    conn.close()

    #Insert into LOCATION TABLE
    conn = sqlite3.connect('cancer.db')
    conn.execute(
        'INSERT INTO LOCATION (Location,User_Id) VALUES (?,?)',
        (location, User_id))
    conn.commit()
    conn.close()

    #Insert into TWEET_URL TABLE
    conn = sqlite3.connect('cancer.db')
    conn.execute(
        'INSERT INTO Tweet_Url (Tweet_Id,Tweet_Url) VALUES (?,?)',
        (Tweet_id, Tweet_Url))
    conn.commit()
    conn.close()

    #Insert into TWEET_MENTION TABLE
    conn = sqlite3.connect('cancer.db')
    for mention in cancer[item]['mentions']:
        conn.execute(
            'INSERT INTO TWEET_MENTION (Source_User,Target_User,Tweet_id) VALUES (?,?,?)',
            (Handle_Name, mention, Tweet_id))
        conn.commit()
    conn.close()

    #Insert into HASHTAG TABLE
    conn = sqlite3.connect('cancer.db')
    for text in cancer[item]['hashtags']:
        conn.execute(
            'INSERT INTO HASHTAG (Hashtag_Text,Tweet_id) VALUES (?,?)',
            (text, Tweet_id))
        conn.commit()
    conn.close()

    #Fetch Hashtag_id from USER TABLE

    Hashtag_id = 0;
    conn = sqlite3.connect('cancer.db')
    cur = conn.cursor()
    cur.execute('SELECT Hashtag_id FROM HASHTAG WHERE Tweet_Id = ? and Hashtag_Text=? collate nocase',
                (Tweet_id, CANCER_TYPE))
    rows = cur.fetchone()

    if rows != None:
        for row in rows:
            Hashtag_id = row

    conn.commit()
    conn.close()

    #Insert into CANCER_TYPE TABLE
    conn = sqlite3.connect('cancer.db')
    conn.execute('INSERT INTO CANCER_TYPE (Hashtag_Id,Cancer_type) VALUES (?,?)', (Hashtag_id, CANCER_TYPE))
    conn.commit()
    conn.close()

    #Fetch Cancer_Id from CANCER_TYPE TABLE
    Cancer_Id = 0;
    conn = sqlite3.connect('cancer.db')
    cur = conn.cursor()
    cur.execute('SELECT Cancer_Id FROM CANCER_TYPE WHERE CANCER_TYPE=?', (CANCER_TYPE,))
    rows = cur.fetchone()

    if rows != None:
        for row in rows:
            Cancer_Id = row

    conn.commit()
    conn.close()

    #Insert into TREATMENT TABLE
    conn = sqlite3.connect('cancer.db')
    treatment = ['Surgery', 'Radiation', 'Chemotherapy']
    conn.execute('INSERT INTO TREATMENT (Cancer_Id,Treatment_type) VALUES (?,?)', (Cancer_Id, random.choice(treatment)))

    conn.commit()
    conn.close()
print("Twitter Data added successfully !")